In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

In [ ]:
folder = '/Users/andreibirladeanu/Documents/Data/play_openpose/1043_play_1'
poses = sorted_alpha(folder)

In [ ]:
frame_no = [int((x.split("."))[0].split("_")[1]) for x in poses]

In [ ]:
def get_skeletons(frame_no, detection):
    """function that reads a detections frame, checks if necks are present, and returns the skeletons"""
    data = pd.read_csv(detection, index_col=0)
    ## first check if it contains the neck 
    temp_data = data
    for i in range(0, len(data.columns),3): # drop skeletons without neck
        if np.isnan(data[data.columns[i]].loc[1]) == True:
            temp_data = temp_data.drop([data.columns[i],data.columns[i+1], data.columns[i+2]], axis=1) ## drop skeletons with missing neck
    data=temp_data
    if len(data.columns) < 3: ## if not enough skeletons with neck present return 0 
        return(None)
    else:
        skeletons=[]
        ## remove probs columns
        for col in data.columns:
            if data[col].name[0:3] == 'pro':
                data=data.drop([col], axis=1)
     # return every skeleton now  
        for x,y, in zip(data.columns[::2], data.columns[1::2]):
            skeletons.append(data[[x,y]].to_numpy())
        return(skeletons) ## this is a list of skeletons(stacked)
        
    

In [ ]:
array_nn = [np.array([110, 100]),np.array([500, 161.28])]
array_fit = [np.array([506.88, 161.28]),  np.array([537.6, 368.64])]
## idea to use the argmin of distance not the index 
neigh = NearestNeighbors(n_neighbors=2, metric='euclidean', algorithm='brute')
neigh.fit(array_nn)
neighb = neigh.kneighbors(array_fit)
neighb

In [ ]:
array_nn = [np.array([500, 161.28]), np.array([110, 100])]
array_fit = [np.array([506.88, 161.28]),  np.array([537.6, 368.64])]
## idea to use the argmin of distance not the index 
neigh = NearestNeighbors(n_neighbors=2, metric='euclidean', algorithm='brute')
neigh.fit(array_nn)
neighb = neigh.kneighbors(array_fit)
neighb

In [ ]:
array_fit = [np.array([110, 100]), np.array([500, 161.28])]
array_nn = [np.array([537.6, 368.64]), np.array([506.88, 161.28]), np.array([400, 300]) ]
neigh = NearestNeighbors(n_neighbors=1, metric='euclidean', algorithm='brute')
distances = []
for elem in array_fit: 
    temp_list = []
    for x in array_nn:
        neigh.fit([x])
        temp_list.append(neigh.kneighbors([elem])[0])
    distances.append(temp_list)
distances

In [ ]:
distances[0]

In [ ]:
from scipy.spatial import distance
distance.euclidean(array_nn, array_fit)

In [ ]:
# case 1. one previous and 2 next 
array_nn = [np.array([110, 100])]
array_fit = [np.array([506.88, 161.28]),  np.array([100, 110])]
neigh = NearestNeighbors(n_neighbors=1, metric='euclidean', algorithm='brute')
neigh.fit(array_nn)
neighb = neigh.kneighbors(array_fit)
neighb
## use the index of the smallest distance but check if it's less than 80 

# case 1. 2 previous and 2 next 
## make sure they aren't close to each other - calculate within distance
## pick the index of the nn 

In [ ]:
np.argmin(neighb[0], axis=1) 

In [ ]:
for dist in range(len(neighb[0])): ## check if they are very close 
    print(neighb[0][dist])

In [ ]:
def nn (prev_detections, new_detect):
    """function that does nn search between the new detections and previous detections"""
    tracklets = [x['skeleton'][-1][1] for x in prev_detections if x['skeleton']!=[]]
    k = 2 if len(tracklets)>1 else 1
    neigh = NearestNeighbors(n_neighbors=2, metric='euclidean', algorithm='brute')
    neigh.fit(tracklets)
    neighbours = neigh.kneighbors([x[1] for x in new_detect], return_distance=True)
    return(neighbours)
    

In [ ]:
def missing_elements(L):
    start, end = L[0], L[-1]
    return sorted(set(range(start, end + 1)).difference(L))

In [ ]:
tracklets = [x['skeleton'][-1][1] for x in tracks if x['skeleton']!=[]]
neigh = NearestNeighbors(n_neighbors=2, metric='euclidean', algorithm='brute')
neigh.fit(tracklets)
neighbours = neigh.kneighbors([x[1] for x in detect], 1, return_distance=True)
print(neighbours)

In [ ]:
track_1, track_2, track_3 = {'frame':[1], 'skeleton':[2]}, {'frame':[4], 'skeleton':[4]}, {'frame':[], 'skeleton':[]}
tracks = [track_1, track_2, track_3]

In [ ]:
track_1, track_2, track_3 = {'frame':[1], 'skeleton':[2]}, {'frame':[4], 'skeleton':[4]}, {'frame':[], 'skeleton':[]}
tracks = [track_1, track_2, track_3]
track_iter = iter(tracks)
for x in 

In [ ]:
track_1, track_2, track_3 = {'frame':[], 'skeleton':[]}, {'frame':[], 'skeleton':[]}, {'frame':[], 'skeleton':[]}
tracks = [track_1, track_2, track_3]
frames = []
for pose, no in zip(poses, frame_no):
    print('frame_no:',no)
    frames.append(frame_no)
    detect = get_skeletons(no, os.path.join(folder, pose))
    if detect == None: continue
    # wait until at least 2 tracks are initialized 
    empty_tracks = 3
    for track in tracks:
        if track['skeleton']!=[]:
            empty_tracks-=1
    if empty_tracks>1:
        for detection in range(len(detect)):
            tracks[detection]['skeleton'].append(detect[detection]) ## in the first frame initalize the tracks
            tracks[detection]['frame'].append(no)
        print(tracks)
        continue
        
    else:
            
        ## nn search
        ## only after at least 2 tracks are initalized 
        
        neighb = nn(tracks, detect) 
        
    #else:
        ### check appearance 
        
    ## interpolation
  #  for track in tracks:
   #     if track['frame']==[] and len(track['frame'])<2:continue
    #    gaps = missing_elements([track['frame'][-1],track['frame'][-2]])
     #   if len(gaps) > 0 and len(gaps) <50:
      #      track['frame'] = insert_position(track['frame'].index(gaps[-1]+1), track['frame'], gaps)
            
    
            

In [ ]:
track_3['frame']

In [ ]:
track_2['frame'][126], track_2['frame'][127]

In [ ]:
## lets try to interpolate one skeleton between two frames:
x = [track_2['frame'][126], track_2['frame'][127]]
y = [track_2['skeleton'][126], track_2['skeleton'][127]]
f = interp1d(x, y, axis=0)

In [ ]:
missing_elements(x)[-1]+1

In [ ]:
to_insert = f(list(range(x[0]+1, x[-1])))

In [ ]:
track_2['frame'].index(track_2['frame'][127])

In [ ]:
def insert_position(position, list1, list2): ## position is the upper bound
    return list1[:position] + list2 + list1[position:]
insert_position(track_2['frame'].index(track_2['frame'][127]), track_2['frame'], missing_elements(x))

In [ ]:
## 1. get the index of the upper bound of the gap 

track_2['frame'].insert(track_2['frame'].index(track_2['frame'][127]), gap[0])

In [ ]:
track_2['frame'] = insert_position(track_2['frame'].index(track_2['frame'][127]), track_2['frame'], missing_elements(x))

In [ ]:
track_2['frame']

In [ ]:
data_1 = pd.read_csv('/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_1.csv')
data_2 = pd.read_csv('/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_2.csv')

In [ ]:
df_1 = get_skeletons(1,'/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_1.csv')
df_2 = get_skeletons(2,'/Users/andrei-macpro/Documents/Data/pose/play_openpose/1043_play_1/frame_2.csv')

In [ ]:
neigh = NearestNeighbors(n_neighbors=1, metric='euclidean')
neigh.fit([x[0][1] for x in tracks if x!=[]])

In [ ]:
neighbours = neigh.kneighbors([x[1] for x in df_2], 1, return_distance=True)
neighbours

In [ ]:
for n in range(len(neighbours)):
    print(neighbours[n])

In [ ]:
f = interp1d(track_1['frame'], track_1['skeleton'],axis=0)

In [ ]:
f([2,5])

In [ ]:
x = [1,5]
y = [[[np.nan,np.nan],[10,11],[12,13]], [[15,16], [17,18], [22.23,20.56]]]
f = interp1d(x, y, axis=0)

In [ ]:
y = [[5,9],[10,11],[0,0],[np.nan,10],[10,11],[12,13], [15,16], [17,18], [19,20]]
mi = pd.MultiIndex.from_arrays([[0,0,0,1,1,1,2,2,2], [1,2,3,1,2,3,1,2,3]], names=['frame', 'joint'])
df = pd.DataFrame(y, columns = ['x','y'], index=mi)

In [ ]:
new_array = f([1,2,3,4,5])


In [ ]:
new_array[-1] = y[-1]

In [ ]:
new_array

In [ ]:
y = [[[5,9],[10,11],[15,16]],[[10,11],[12,13], [np.nan, np.nan]], [[17,18], [19,20],[7,8]],
    [[10,11], [0,8], [9,8]], [[20,4], [19,8], [10,5]]]

savgol_filter(np.array(y), 5, 3, axis=0,mode='interp')

In [ ]:
y = [[[5,9],[10,11],[0,0],[np.nan,10]],[[10,11],[12,13], [15,16]], [[17,18], [19,20],[0,5]],
    [[10,11], [0,8], [9,8]], [[20,4], [19,8], [10,5]]]
np.array(y).shape

In [ ]:
y

In [ ]:
x = 'nan'
x!=x